In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
import joblib
import re
from PIL import Image
# import dlib
import matplotlib.pyplot as plt
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.python.client import device_lib
import sklearn
from sklearn.model_selection import train_test_split
from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.linear_model import Lasso,LogisticRegression
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline

In [4]:
dataset= joblib.load("D:/Down/dataset_big3.pkl")

In [3]:
path = "D:/Down/inha_data (2)/inha_data"
os.chdir(path)

In [12]:
keras_reg64 = tf.saved_model.load("keras_reg")

In [6]:
y = dataset['label']
X = pd.DataFrame(dataset['data'])
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size =.3, random_state = 0)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=.3, random_state=0)

In [23]:
# sklearn.metrics.SCORERS.keys()

dict_keys(['explained_variance', 'r2', 'max_error', 'neg_median_absolute_error', 'neg_mean_absolute_error', 'neg_mean_absolute_percentage_error', 'neg_mean_squared_error', 'neg_mean_squared_log_error', 'neg_root_mean_squared_error', 'neg_mean_poisson_deviance', 'neg_mean_gamma_deviance', 'accuracy', 'top_k_accuracy', 'roc_auc', 'roc_auc_ovr', 'roc_auc_ovo', 'roc_auc_ovr_weighted', 'roc_auc_ovo_weighted', 'balanced_accuracy', 'average_precision', 'neg_log_loss', 'neg_brier_score', 'adjusted_rand_score', 'rand_score', 'homogeneity_score', 'completeness_score', 'v_measure_score', 'mutual_info_score', 'adjusted_mutual_info_score', 'normalized_mutual_info_score', 'fowlkes_mallows_score', 'precision', 'precision_macro', 'precision_micro', 'precision_samples', 'precision_weighted', 'recall', 'recall_macro', 'recall_micro', 'recall_samples', 'recall_weighted', 'f1', 'f1_macro', 'f1_micro', 'f1_samples', 'f1_weighted', 'jaccard', 'jaccard_macro', 'jaccard_micro', 'jaccard_samples', 'jaccard_wei

array([[1., 0.],
       [0., 1.],
       [0., 1.],
       ...,
       [0., 1.],
       [1., 0.],
       [1., 0.]], dtype=float32)

In [22]:
# C=1, 5, 10, 50, 100
lg_clf = LogisticRegression()

In [26]:
lg_param = [
    {
        'C' : [1, 10, 50, 100]
    }
]
lg_grid = RandomizedSearchCV(lg_clf, lg_param, n_iter=10, cv =5, verbose =100, n_jobs = -1)
lg_grid.fit(X_train, y_train)

C:\Users\pc\Anaconda3\envs\tensorflow2_2\lib\site-packages\sklearn\model_selection\_search.py:289: UserWarning: The total space of parameters 4 is smaller than n_iter=10. Running 4 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)


Fitting 5 folds for each of 4 candidates, totalling 20 fits


C:\Users\pc\Anaconda3\envs\tensorflow2_2\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


RandomizedSearchCV(cv=5, estimator=LogisticRegression(C=100), n_jobs=-1,
                   param_distributions=[{'C': [1, 10, 50, 100]}], verbose=100)

In [29]:
yp = lg_grid.predict_proba(X_val)[:,1]
m = tf.keras.metrics.AUC()
m.update_state(y_val, yp)
m.result().numpy()

0.9906537

In [57]:
a =keras_reg(X_val.loc[:,:].values)

In [62]:
a.numpy().reshape((-1,))

(364869,)

In [63]:
estimators = [lg_clf, keras_reg]

X_val_predictions = np.empty((len(X_val), len(estimators)), dtype=np.float32)
X_val_predictions[:,0] = estimators[0].predict_proba(X_val)[:,1]
X_val_predictions[:,1] = estimators[1](X_val.values).numpy().reshape((-1,))


In [77]:
rnd_forest_blender = RandomForestClassifier( oob_score=True,  max_features='sqrt')

rnd_param = [
    {
        "n_estimators" : [100,300, 500],
        "max_depth" : [5,10]
    }
    
]
rnd_grid = RandomizedSearchCV(rnd_forest_blender, rnd_param, n_iter=10, cv=5, verbose =100)

In [78]:
rnd_grid.fit(X_val_predictions, y_val)
# rnd_forest_blender.fit(X_val_predictions, y_val)


C:\Users\pc\Anaconda3\envs\tensorflow2_2\lib\site-packages\sklearn\model_selection\_search.py:289: UserWarning: The total space of parameters 6 is smaller than n_iter=10. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)


Fitting 5 folds for each of 6 candidates, totalling 30 fits
[CV 1/5; 1/6] START max_depth=5, n_estimators=100...............................
[CV 1/5; 1/6] END max_depth=5, n_estimators=100;, score=0.960 total time=  16.7s
[CV 2/5; 1/6] START max_depth=5, n_estimators=100...............................
[CV 2/5; 1/6] END max_depth=5, n_estimators=100;, score=0.961 total time=  16.8s
[CV 3/5; 1/6] START max_depth=5, n_estimators=100...............................
[CV 3/5; 1/6] END max_depth=5, n_estimators=100;, score=0.962 total time=  16.9s
[CV 4/5; 1/6] START max_depth=5, n_estimators=100...............................
[CV 4/5; 1/6] END max_depth=5, n_estimators=100;, score=0.961 total time=  16.8s
[CV 5/5; 1/6] START max_depth=5, n_estimators=100...............................
[CV 5/5; 1/6] END max_depth=5, n_estimators=100;, score=0.961 total time=  16.9s
[CV 1/5; 2/6] START max_depth=5, n_estimators=300...............................
[CV 1/5; 2/6] END max_depth=5, n_estimators=300;,

RandomizedSearchCV(cv=5,
                   estimator=RandomForestClassifier(max_features='sqrt',
                                                    n_estimators=200,
                                                    oob_score=True,
                                                    random_state=42),
                   param_distributions=[{'max_depth': [5, 10],
                                         'n_estimators': [100, 300, 500]}],
                   verbose=100)

In [79]:
rnd_grid.best_params_

{'n_estimators': 100, 'max_depth': 5}

In [70]:
X_test_predictions = np.empty((len(X_test), len(estimators)))
X_test_predictions[:,0] = estimators[0].predict_proba(X_test)[:,1]
X_test_predictions[:,1] = estimators[1](X_test.values).numpy().reshape((-1,))

In [80]:
yp = rnd_grid.predict(X_test_predictions)
m = tf.keras.metrics.AUC()
m.update_state(y_test, yp)
m.result().numpy()

0.9615404

In [82]:
yp = lg_grid.predict_proba(X_test)[:,1]
m = tf.keras.metrics.AUC()
m.update_state(y_test, yp)
m.result().numpy()

0.9905473

In [84]:
yp = keras_reg(X_test.values)
m = tf.keras.metrics.AUC()
m.update_state(y_test, yp)
m.result().numpy()

0.9922337